# Creating the HTLC-Success Transaction

In this section, we'll build a Lightning channel HTLC-success transaction from scratch using Python. We'll walk through each part of the transaction — how it's constructed and signed. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we'll use Bob's commitment transaction created in `workshop-solution-chapter3` which contains the accepted HTLC output that we'll spend with the HTLC-success transaction.

In [1]:
%run "../chapter 3 - in-flight htlc commitment transaction/workshop-solution-chapter3.ipynb"

2026-02-13T19:31:13.221000Z TestFramework (INFO): PRNG seed is: 3218702974398088220
2026-02-13T19:31:13.223000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_km2dxaot
🟢 New TestShell started. Block height: 0
Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Payment BasePoint:  025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Bob Payment BasePoint:  02f98efd3f2b2fbe7bd83c419f5f64f8280798b8a9175fdb77c0091bbb95c79506
To obscure commitment number 0xb433fd43a66f
Alice funding pubkey: 0e05e6f016f05d96ed800f36288d94f8e8e27df7d4d563c5a82480f6f892b436
Alice funding privkey: a1b5f330270bcb68f0df3b3a0fd321fac4cb30d85d774681017fe1ed8e251df1
Alice funding address: bcrt1ppcz7duqk7pwedmvqpumz3rv5lr5wyl0h6n2k83dgyjq0d7yjksmq3m4hxk
Alice sweeper pubkey: 084539657b96e28d30fae99d130606ab9c63077d32755a57ec62fc3507bc5a0b
Alice s

## HTLC-Success Transaction

The HTLC-success transaction is a second-level transaction that spends an accepted HTLC output from a commitment transaction. It allows Bob (who received the HTLC) to claim the funds by providing the payment preimage before the timeout period expires.

**Key differences from HTLC-timeout:**
- Spends an **accepted HTLC** (Bob's perspective) instead of offered HTLC
- Requires the **payment preimage** to unlock
- Uses scriptB (success path) instead of scriptA (timeout path)
- Must be claimed before CLTV timeout expires

### The Unsigned HTLC-Success Transaction

#### The Input

The input is the accepted HTLC output from Bob's commitment transaction (output index 2).

In [2]:
# Get the commitment transaction details
# Bob's commitment transaction has the accepted HTLC output at index 2
commitment_txid = decoded['txid']
commitment_output_index = 2

print(f"Spending from Bob's commitment tx: {commitment_txid}")
print(f"Output index: {commitment_output_index} (accepted HTLC)")

# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
commitment_txid_bytes = bytes.fromhex(commitment_txid)[::-1]
commitment_index = commitment_output_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# Sequence: must be set to 1 for the CSV delay in scriptB
# scriptB requires: OP_1 OP_CHECKSEQUENCEVERIFY
sequence_htlc = bytes.fromhex("01000000")

inputs = (
    commitment_txid_bytes
    + commitment_index
    + varint_len(scriptsig)
    + scriptsig
    + sequence_htlc
)

Spending from Bob's commitment tx: c76deacb2c41bf2bc5e38a9e7d9f8a08fcb51f6c2b68956e22b2b4a0e8f186d1
Output index: 2 (accepted HTLC)


#### The Output

The HTLC-success transaction has a single output that pays to Bob with a delay. This output is similar to the `to_local` output in commitment transactions: it's spendable by Bob after a delay, or immediately by Alice if she has the revocation key (in case this commitment gets revoked).

#### To Local Delayed Output

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +-------------------+
                    ---| P(revocation) + T |
                       +-------------------+
                                         ^
                                         |
                                   +-----------+        
                                   | T = t * G |
                                   +-----------+        
                                         ^
                                         |
     +---+   +-------------------------------------------------------+
     | t | = | TaggedHash ("Taptweak", P(revocation) || script_root) |
     +---+   +-------------------------------------------------------+
                                                             ^
                                                             |
                                                          +-----+
                                                          |  h  |
                                                          +-----+                   
                                                             ^                                                          
                                                             |                                                          
      +-----------------------------------------------------------+                                           
      | P(local_delayed) OP_CHECKSIG to_self_delay OP_CSV OP_DROP |                                           
      +-----------------------------------------------------------+

**Note:** If option_anchors applies (which is the case here), then the HTLC-success transactions are signed with the input and output having the same value. This means they have a zero fee and MUST be combined with other inputs to arrive at a reasonable fee.

In [3]:
# Output count
output_count = bytes.fromhex("01")

# Output value (same as HTLC input value for zero-fee transaction)
output_value_sat = htlc_output_value_satoshis
output_value = output_value_sat.to_bytes(8, byteorder="little", signed=False)

# We already have bob_delayed_pubkey and alice_revocation_pubkey from chapter 3
# These are the keys used for Bob's outputs

# Create the script: P(local_delayed) OP_CHECKSIG to_self_delay OP_CSV OP_DROP
to_self_delay = 144
script_output = CScript([bob_delayed_pubkey.get_bytes(bip340=True), OP_CHECKSIG, to_self_delay, OP_CHECKSEQUENCEVERIFY, OP_DROP])

# Compute output script_root
hash_input = TAPSCRIPT_VER + ser_string(script_output)
script_root = tagged_hash("TapBranch", hash_input)

# Compute the output Tagged Hash (using Alice's revocation key since Bob holds her commitment)
taptweak = tagged_hash("TapTweak", alice_revocation_pubkey.get_bytes(bip340=True) + script_root)
htlc_alice_revocation_pubkey_tweaked = alice_revocation_pubkey.tweak_add(taptweak)

# Compute scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output_spk = bytes.fromhex("51") + varint_len(htlc_alice_revocation_pubkey_tweaked.get_bytes(bip340=True)) + htlc_alice_revocation_pubkey_tweaked.get_bytes(bip340=True)

outputs = (
    output_value
    + varint_len(output_spk)
    + output_spk
)

# Locktime: set to 0 (no CLTV requirement for success path)
locktime = bytes.fromhex("00000000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

print("\nunsigned_tx:", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded_htlc_success = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded_htlc_success, indent=2, default=str))


unsigned_tx: 0200000001d186f1e8a0b4b2226e95682b6c1fb5fc088a9f7d9e8ae3c52bbf412ccbea6dc70200000000010000000120a107000000000022512042007f9cdbb3bca83f92d0bfae2b4f863f1fac14612a6dd3d5785a27469c9ec200000000
{
  "txid": "11990b192f6d3d85fbfd4d0e8b60d5e1d049301daabed1350ff33d40c03cfd5f",
  "hash": "11990b192f6d3d85fbfd4d0e8b60d5e1d049301daabed1350ff33d40c03cfd5f",
  "version": 2,
  "size": 94,
  "vsize": 94,
  "weight": 376,
  "locktime": 0,
  "vin": [
    {
      "txid": "c76deacb2c41bf2bc5e38a9e7d9f8a08fcb51f6c2b68956e22b2b4a0e8f186d1",
      "vout": 2,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 1
    }
  ],
  "vout": [
    {
      "value": "0.00500000",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 42007f9cdbb3bca83f92d0bfae2b4f863f1fac14612a6dd3d5785a27469c9ec2",
        "desc": "rawtr(42007f9cdbb3bca83f92d0bfae2b4f863f1fac14612a6dd3d5785a27469c9ec2)#f5xwp037",
        "hex": "512042007f9cdbb3bca83f92d0bfae2b4f863f1fac14612a6dd3d5785a27

## The sighash for script path spend

Unlike the commitment transaction which uses **key path spending**, the HTLC-success transaction uses **script path spending** through the Taproot tree. We need to spend using scriptB of the accepted HTLC output.

The sighash calculation for script path spending includes additional data about the script being executed.

In [4]:
# We're spending the accepted HTLC output using scriptB (success path)
spending_script = scriptB

# Calculate the tapleaf hash for the script we're spending
hash_input_spending = TAPSCRIPT_VER + ser_string(spending_script)
tapleaf_hash = tagged_hash("TapLeaf", hash_input_spending)
key_version = bytes.fromhex("00")  # reserved for future upgrades
codeseparator = bytes.fromhex("ffffffff")  

# SIGHASH for script path spend (BIP-341)
index_of_this_input = bytes.fromhex("0000 0000")
sighash_epoch = bytes.fromhex("00")
hash_type = bytes.fromhex("83")  # SIGHASH_SINGLE|SIGHASH_ANYONECANPAY (0x03 | 0x80)

# When SIGHASH_SINGLE (0x03) is set, sha_outputs is 32 bytes (BIP-341)
sha_outputs = sha256(outputs).digest()

# Data about this input, script path (no annex)
spend_type = bytes.fromhex("02")

# Common signature message extension (BIP-341)
# When SIGHASH_ANYONECANPAY is set, we include data about THIS specific input:
outpoint = commitment_txid_bytes + commitment_index  # 36 bytes
amount = htlc_output_value  # 8 bytes
nSequence = sequence_htlc  # 4 bytes

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + spend_type
    + outpoint          
    + amount
    + varint_len(bob_accepted_htlc_spk)
    + bob_accepted_htlc_spk
    + nSequence
    + sha_outputs
    + tapleaf_hash
    + key_version
    + codeseparator
)

print("\nSignature message:", sig_msg.hex())

tag_hash = sha256("TapSighash".encode()).digest()
sighash = sha256(tag_hash + tag_hash + sig_msg).digest()
print("\nSighash:", sighash.hex())


Signature message: 0083020000000000000002d186f1e8a0b4b2226e95682b6c1fb5fc088a9f7d9e8ae3c52bbf412ccbea6dc70200000020a10700000000002251204cd5b607b8c1680cbd82425ff0029716c3ce0269155e03b8b5d3af14a4c722bb010000002b70f8a436eb1acd614d30128ada59f52ad67881820e5c847294d65d359faea0dc2dea3000cdf9c51e3ffdd8656dad6e877b1b033611ac0335384d9b0cdbfa7300ffffffff

Sighash: 7f3823d2d45564361c7c0e94365930f8c4cf639203936bc1d3395ae76715eeb4


## Signing the sighash

For the HTLC-success transaction, we need signatures from both Bob's and Alice's HTLC keys. These are regular Schnorr signatures (not MuSig2 aggregated signatures), since the script requires two separate signature checks.

In [5]:
# Derive Bob's HTLC private key (local for Bob)
bob_htlc_basepoint = derivate_key(bob_node_seed, family=2, channel_index=0)
bob_htlc_privkey = bob_htlc_basepoint.get_privkey(bob_per_commitment.get_pub())

# Derive Alice's HTLC private key (remote for Bob)
alice_htlc_basepoint = derivate_key(alice_node_seed, family=2, channel_index=0)
alice_htlc_privkey = alice_htlc_basepoint.get_privkey(alice_per_commitment.get_pub())

# Generate auxiliary random data for each signature
import secrets
aux_bob = secrets.token_bytes(32)
aux_alice = secrets.token_bytes(32)

# Sign with Bob's HTLC key (for OP_CHECKSIGVERIFY)
bob_htlc_sig = bob_htlc_privkey.sign_schnorr(sighash, aux_bob)
# APPEND sighash type byte for non-default sighash!
bob_htlc_sig = bob_htlc_sig + hash_type  # Add 0x83 byte

# Verify Bob's signature (without the sighash byte for verification)
bob_sig_valid = bob_htlc_pubkey.verify_schnorr(bob_htlc_sig[:-1], sighash)
print(bob_htlc_sig.hex())
print("Bob signature valid?", bob_sig_valid)
print(f"Bob signature length: {len(bob_htlc_sig)} bytes (should be 65)")

# Sign with Alice's HTLC key (for OP_CHECKSIG)
alice_htlc_sig = alice_htlc_privkey.sign_schnorr(sighash, aux_alice)
# APPEND sighash type byte for non-default sighash!
alice_htlc_sig = alice_htlc_sig + hash_type  # Add 0x83 byte

# Verify Alice's signature (without the sighash byte for verification)
alice_sig_valid = alice_htlc_pubkey.verify_schnorr(alice_htlc_sig[:-1], sighash)
print(alice_htlc_sig.hex())
print("Alice signature valid?", alice_sig_valid)
print(f"Alice signature length: {len(alice_htlc_sig)} bytes (should be 65)")

7290a373978acf94586ee98a8d2dd8a8108e87113e00c408018e45e79aa839752a3a209ac699bf16181107400858f5122abbcc519b4e6dd43472001367fd395283
Bob signature valid? True
Bob signature length: 65 bytes (should be 65)
989d794151adc80c634f8a17eabec99eb99d12adfe6fffd53a36bce5b0663f5edf21f63989a8eacd0ebc304ad751969209cb309deeb7a909094719f93d57dc0d83
Alice signature valid? True
Alice signature length: 65 bytes (should be 65)


## The signed transaction

Now we construct the witness for the script path spend. The witness stack for spending scriptB (success path) contains:
1. Alice's signature (last on stack, for OP_CHECKSIG)
2. Bob's signature (for OP_CHECKSIGVERIFY)
3. The payment preimage (to satisfy the hash check)
4. The script being executed (scriptB)
5. The control block (proving scriptB is in the taproot tree)

In [6]:
# Construct the control block
# Control block format: <version byte> <internal key> <parity bit> [<merkle proof>]
# Version byte: 0xc0 | (parity of Q)

# Get parity bit from Q
parity = 0 if alice_revocation_pubkey_tweaked.get_bytes(bip340=False)[0] == 0x02 else 1
version_byte = bytes([0xc0 | parity])

# The control block includes the merkle proof (taggedhash_leafA since we're spending leafB)
control_block = version_byte + alice_revocation_pubkey.get_bytes(bip340=True) + htlc_taggedhash_leafA

print("\ncontrol_block:", control_block.hex())
print("\nspending_script:", spending_script.hex())
print("\npayment_preimage:", payment_preimage.hex())

# Construct witness stack
# Stack order (bottom to top): <preimage> <bob_sig> <alice_sig> <script> <control_block>
witness = (
    bytes.fromhex("05")  # 5 stack items
    + varint_len(alice_htlc_sig)    # Alice's signature (for OP_CHECKSIG)
    + alice_htlc_sig
    + varint_len(bob_htlc_sig)      # Bob's signature (for OP_CHECKSIGVERIFY)
    + bob_htlc_sig
    + varint_len(payment_preimage)  # The payment preimage
    + payment_preimage
    + varint_len(spending_script)   # The script
    + spending_script
    + varint_len(control_block)     # The control block
    + control_block
)

# The final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("\nsigned tx:", signed_tx.hex())

# Before testing it, we need to confirm the commitment transaction so that the HTLC output exists on-chain
bob_commitment_txid = node.sendrawtransaction(signed_bob_commitment_tx.hex())
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

print("bob_commitment_txid: ", bob_commitment_txid)

# Now we can test if the HTLC timeout transaction would be accepted in the mempool (it shouldn't, since it is paying no fee)
print("\nTest mempool accept:")
try:
    result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
    print(json.dumps(result, indent=2, default=str))
except Exception as e:
    print(f"Expected error (commitment tx not in mempool): {e}")


control_block: c102c3bcb049c8059227f85f26805179b4ff6e0573072696e4740f59cc0e21e37fa37dde69333f309b48c6d376f747b10502e8c35ae31714d6f041945f48ee7558

spending_script: 82012088a91413b40d560301d7b55358acf23bd6dfdde738260188207bd8383df06640c0efe760bb2bac1a1585bdc664e6f1cb61c41a76a6f2be3484ad207546c6ab3a8c005ce24e9feb7154fd35b62ac0f9125beb5e692d4d912a95de77ac

payment_preimage: fb53d94b7b65580f75b98f100d0345da08c03521bdab6d519143cd521d1b3826

signed tx: 02000000000101d186f1e8a0b4b2226e95682b6c1fb5fc088a9f7d9e8ae3c52bbf412ccbea6dc70200000000010000000120a107000000000022512042007f9cdbb3bca83f92d0bfae2b4f863f1fac14612a6dd3d5785a27469c9ec20541989d794151adc80c634f8a17eabec99eb99d12adfe6fffd53a36bce5b0663f5edf21f63989a8eacd0ebc304ad751969209cb309deeb7a909094719f93d57dc0d83417290a373978acf94586ee98a8d2dd8a8108e87113e00c408018e45e79aa839752a3a209ac699bf16181107400858f5122abbcc519b4e6dd43472001367fd39528320fb53d94b7b65580f75b98f100d0345da08c03521bdab6d519143cd521d1b38265f82012088a91413b40d560301d7b553

## Adding input for fees

As stated in the BOLTs: "If option_anchors applies, then the HTLC-timeout and HTLC-success transactions are signed with the input and output having the same value. This means they have a zero fee and MUST be combined with other inputs to arrive at a reasonable fee."

We'll now add Bob's sweeper output created in chapter 2 as a second input to pay for fees.

In [7]:
# INPUTS
# We have 2 inputs: the HTLC output and bob sweeper output
input_count_v2 = bytes.fromhex("02")

# INPUT 2: bob sweeper output from chapter 2
# Get Bob's sweeper transaction details from chapter 2
bob_txid_to_spend_bytes = bytes.fromhex(txid_to_spend)[::-1]
bob_sweeper_index_bytes = (bob_sweeper_index).to_bytes(4, byteorder="little", signed=False)
sequence_change = bytes.fromhex("ffffffff")

inputs_v2 = (
    commitment_txid_bytes
    + commitment_index
    + varint_len(scriptsig)
    + scriptsig
    + sequence_htlc
    + bob_txid_to_spend_bytes
    + bob_sweeper_index_bytes
    + varint_len(scriptsig)
    + scriptsig
    + sequence_change
)

print(f"Added bob_sweeper input from funding tx: {funding_channel_txid}:{bob_sweeper_index}")

Added bob_sweeper input from funding tx: 6f7689e888a648d90b93dca78fe28b3ab1d8e7e5a1a9aefd5789ce3b48b48ea6:0


In [8]:
# OUTPUTS
# Now we have 2 outputs: HTLC delayed output + change back to bob
output_count_v2 = bytes.fromhex("02")

# OUTPUT 1: HTLC delayed output (same as before)
# Reuse the same output_value and output_spk from v1

# OUTPUT 2: Change back to bob_change address
# Calculate: bob_sweeper_value (98,999,700) + htlc_value (500,000) - htlc_output (500,000) - fee
tx_fee_sat = 300
bob_change_value = int(sweeper_initial_fund * 100000000) - tx_fee_sat
bob_change_value_bytes = bob_change_value.to_bytes(8, byteorder="little", signed=False)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + bob_change_pubkey
output_spk_change = bytes.fromhex("51") + varint_len(bob_change_pubkey.get_bytes(bip340=True)) + bob_change_pubkey.get_bytes(bip340=True)

# Use outputs_v2 to avoid overwriting the original outputs variable
outputs_v2 = (
    output_value
    + varint_len(output_spk)
    + output_spk
    + bob_change_value_bytes
    + varint_len(output_spk_change)
    + output_spk_change
)

print(f"Transaction fee: {tx_fee_sat} sats")

Transaction fee: 300 sats


In [9]:
unsigned_tx_v2 = (
    version
    + input_count_v2
    + inputs_v2
    + output_count_v2
    + outputs_v2
    + locktime
)

print("unsigned_tx_v2:", unsigned_tx_v2.hex())

# Decode the unsigned transaction to verify it looks correct
decoded_htlc_success_v2 = node.decoderawtransaction(unsigned_tx_v2.hex())
print(json.dumps(decoded_htlc_success_v2, indent=2, default=str))

unsigned_tx_v2: 0200000002d186f1e8a0b4b2226e95682b6c1fb5fc088a9f7d9e8ae3c52bbf412ccbea6dc70200000000010000004a22a25652b874ae8d3f144e0fa010e57fdc28f91e505ee4f01ddca34876bf960000000000ffffffff0220a107000000000022512042007f9cdbb3bca83f92d0bfae2b4f863f1fac14612a6dd3d5785a27469c9ec25495980000000000225120b9a580d4f595ca4beebd1fc09168503dc8e6c299a72069833c5c1dac0714d80e00000000
{
  "txid": "0278e92d91df333481bd21491776d74e5661f013cfa8036a031558a234946ab1",
  "hash": "0278e92d91df333481bd21491776d74e5661f013cfa8036a031558a234946ab1",
  "version": 2,
  "size": 178,
  "vsize": 178,
  "weight": 712,
  "locktime": 0,
  "vin": [
    {
      "txid": "c76deacb2c41bf2bc5e38a9e7d9f8a08fcb51f6c2b68956e22b2b4a0e8f186d1",
      "vout": 2,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 1
    },
    {
      "txid": "96bf7648a3dc1df0e45e501ef928dc7fe510a00f4e143f8dae74b85256a2224a",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      

### Sign the bob sweeper input (Input 1 - key path spend)

The HTLC input signatures were already created earlier and can be reused. We only need to sign the bob_sweeper input using key path spending.

In [10]:
# SIGHASH for key path spend
sighash_epoch = bytes.fromhex("00")
hash_type_keypath = bytes.fromhex("00")  # SIGHASH_DEFAULT (SIGHASH_ALL)

# Transaction data - now includes both inputs
bob_sweeper_value = int(sweeper_initial_fund * 100000000)
bob_sweeper_value_bytes = bob_sweeper_value.to_bytes(8, byteorder="little", signed=False)
sha_prevouts_v2 = sha256(commitment_txid_bytes + commitment_index + bob_txid_to_spend_bytes + bob_sweeper_index_bytes).digest()
sha_amounts_v2 = sha256(htlc_output_value + bob_sweeper_value_bytes).digest()

# scriptPubKeys for both inputs
bob_accepted_htlc_spk_with_len = varint_len(bob_accepted_htlc_spk) + bob_accepted_htlc_spk
bob_sweeper_spk = bytes.fromhex("51") + varint_len(bob_sweeper_pubkey.get_bytes(bip340=True)) + bob_sweeper_pubkey.get_bytes(bip340=True)
bob_sweeper_spk_with_len = varint_len(bob_sweeper_spk) + bob_sweeper_spk
sha_scriptpubkeys_v2 = sha256(bob_accepted_htlc_spk_with_len + bob_sweeper_spk_with_len).digest()

sha_sequences_v2 = sha256(sequence_htlc + sequence_change).digest()
sha_outputs_v2 = sha256(outputs_v2).digest()

# Data about this input (key path spend)
spend_type = bytes.fromhex("00")  # key path, no annex

# Index of the input being signed (input 1, the sweeper input)
input_index = bytes.fromhex("01000000")

sig_msg_sweeper = (
    sighash_epoch
    + hash_type_keypath
    + version
    + locktime
    + sha_prevouts_v2
    + sha_amounts_v2
    + sha_scriptpubkeys_v2
    + sha_sequences_v2
    + sha_outputs_v2
    + spend_type
    + input_index
)

tag_hash = sha256("TapSighash".encode()).digest()
sighash_sweeper = sha256(tag_hash + tag_hash + sig_msg_sweeper).digest()

# Sign with bob_change_privkey
import secrets
aux_sweeper = secrets.token_bytes(32)
bob_sweeper_sig = bob_sweeper_privkey.sign_schnorr(sighash_sweeper, aux_sweeper)

# Verify the signature
bob_sweeper_sig_valid = bob_sweeper_pubkey.verify_schnorr(bob_sweeper_sig, sighash_sweeper)
print("Bob sweeper signature valid?", bob_sweeper_sig_valid)

Bob sweeper signature valid? True


### Build the final signed transaction

Now we construct the witness data for both inputs and build the final signed transaction.

In [11]:
# Witness for Input 1 (bob change) - key path spend
witness_input1 = (
    bytes.fromhex("01")  # 1 stack item
    + varint_len(bob_sweeper_sig)
    + bob_sweeper_sig
)

# Complete witness data (HTLC witness + sweeper witness)
witness_v2 = witness + witness_input1

# The final signed transaction
signed_tx_v2 = (
    version
    + marker
    + flag
    + input_count_v2
    + inputs_v2
    + output_count_v2
    + outputs_v2
    + witness_v2
    + locktime
)

print("signed tx v2:", signed_tx_v2.hex())

# Decode the signed transaction
decoded_signed_v2 = node.decoderawtransaction(signed_tx_v2.hex())
print("\n" + json.dumps(decoded_signed_v2, indent=2, default=str))

signed tx v2: 02000000000102d186f1e8a0b4b2226e95682b6c1fb5fc088a9f7d9e8ae3c52bbf412ccbea6dc70200000000010000004a22a25652b874ae8d3f144e0fa010e57fdc28f91e505ee4f01ddca34876bf960000000000ffffffff0220a107000000000022512042007f9cdbb3bca83f92d0bfae2b4f863f1fac14612a6dd3d5785a27469c9ec25495980000000000225120b9a580d4f595ca4beebd1fc09168503dc8e6c299a72069833c5c1dac0714d80e0541989d794151adc80c634f8a17eabec99eb99d12adfe6fffd53a36bce5b0663f5edf21f63989a8eacd0ebc304ad751969209cb309deeb7a909094719f93d57dc0d83417290a373978acf94586ee98a8d2dd8a8108e87113e00c408018e45e79aa839752a3a209ac699bf16181107400858f5122abbcc519b4e6dd43472001367fd39528320fb53d94b7b65580f75b98f100d0345da08c03521bdab6d519143cd521d1b38265f82012088a91413b40d560301d7b55358acf23bd6dfdde738260188207bd8383df06640c0efe760bb2bac1a1585bdc664e6f1cb61c41a76a6f2be3484ad207546c6ab3a8c005ce24e9feb7154fd35b62ac0f9125beb5e692d4d912a95de77ac41c102c3bcb049c8059227f85f26805179b4ff6e0573072696e4740f59cc0e21e37fa37dde69333f309b48c6d376f747b10502e8c35ae3

In [12]:
# Test mempool accept
print("\nTest mempool accept:")
try:
    result = node.testmempoolaccept(rawtxs=[signed_tx_v2.hex()])
    print(json.dumps(result, indent=2, default=str))
    
    if result[0]['allowed']:
        print("\n✓ Transaction is valid and meets minimum relay fee!")
        print(f"  Fee: {result[0].get('fees', {}).get('base', 'N/A')} BTC")
    else:
        print(f"\n✗ Transaction rejected: {result[0].get('reject-reason', 'Unknown')}")
except Exception as e:
    print(f"Error: {e}")


Test mempool accept:
[
  {
    "txid": "0278e92d91df333481bd21491776d74e5661f013cfa8036a031558a234946ab1",
    "wtxid": "52d8b2bfae72af84a209c1c6fca50769b62175e134cc4a6ced00a27cb0cc7b61",
    "allowed": true,
    "vsize": 277,
    "fees": {
      "base": "0.00000300",
      "effective-feerate": "0.00001083",
      "effective-includes": [
        "52d8b2bfae72af84a209c1c6fca50769b62175e134cc4a6ced00a27cb0cc7b61"
      ]
    }
  }
]

✓ Transaction is valid and meets minimum relay fee!
  Fee: 0.00000300 BTC


## Summary

In this chapter, we successfully created an HTLC-success transaction that spends the accepted HTLC output from Bob's commitment transaction. Key points:

1. **Purpose**: The HTLC-success transaction allows Bob to claim funds from an accepted HTLC by providing the payment preimage.

2. **Script Path Spending**: Uses script path spending through the Taproot tree, specifically scriptB (success path).

3. **Payment Preimage**: The critical component - Bob must provide the preimage that hashes to the payment_hash.

4. **Two Signatures Required**: The success script requires signatures from both Bob's and Alice's HTLC keys.

5. **Delayed Output**: The output is similar to `to_local` - spendable by Bob after a delay (to_self_delay), or immediately by Alice if she has the revocation key.

6. **Witness Structure**: The witness includes both signatures, the payment preimage, the script, and a control block proving the script is part of the Taproot tree.

7. **Zero Fee**: Like HTLC-timeout, this transaction has zero fee and must be combined with additional inputs to pay for mining fees.

8. **Key Difference from HTLC-Timeout**: 
   - HTLC-timeout: Alice reclaims after timeout (scriptA)
   - HTLC-success: Bob claims with preimage (scriptB)